<h3>Importing thge neccessary libraries for completing the project<h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy.stats import zscore


<h1>Data Loading and Cleaning<h1>

<h3>Data loading and displaying the first few rows. This will help to give an idea of the columns and values that are being dealt with.<h3>

In [2]:
df = pd.read_csv('/workspaces/Practise-Code/Kaggle Project/Data/Raw Data/kc_house_data.csv').sort_values('price', ascending=False)
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
7252,6762700020,20141013T000000,7700000.0,6,8.00,12050,27600,2.5,0,3,...,13,8570,3480,1910,1987,98102,47.6298,-122.323,3940,8800
3914,9808700762,20140611T000000,7062500.0,5,4.50,10040,37325,2.0,1,2,...,11,7680,2360,1940,2001,98004,47.6500,-122.214,3930,25449
9254,9208900037,20140919T000000,6885000.0,6,7.75,9890,31374,2.0,0,4,...,13,8860,1030,2001,0,98039,47.6305,-122.240,4540,42730
4411,2470100110,20140804T000000,5570000.0,5,5.75,9200,35069,2.0,0,0,...,13,6200,3000,2001,0,98039,47.6289,-122.233,3560,24345
1448,8907500070,20150413T000000,5350000.0,5,5.00,8000,23985,2.0,0,4,...,12,6720,1280,2009,0,98004,47.6232,-122.220,4600,21750


<h3>Using '.info()' to give us an idea of missing values and the data types used.<h3>

In this instance, the data is not missing values and most columns are using numeric data types allowing for easier analysis (catagorical values such as 'view' or 'seafront' have been converted to numerical already). Date however could benefit from being converted to 'datetime' which will allow for more control when generating EDA (Exploritory Data Analysis).

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21613 entries, 7252 to 1149
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int64  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int64  
 6   sqft_lot       21613 non-null  int64  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int64  
 9   view           21613 non-null  int64  
 10  condition      21613 non-null  int64  
 11  grade          21613 non-null  int64  
 12  sqft_above     21613 non-null  int64  
 13  sqft_basement  21613 non-null  int64  
 14  yr_built       21613 non-null  int64  
 15  yr_renovated   21613 non-null  int64  
 16  zipcode        21613 non-null  int64  
 17  lat            21613 non-null  float64
 18  long     

<h3>Below demonstrates the conversion of the 'date' column to 'datetime' and how the type of data for 'date' changes when we call 'info()' again.<h3>

In [4]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21613 entries, 7252 to 1149
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  int64         
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   21613 n

<h3>Outliers are handled below by creating Z scores for all data and making sure they fall wihtin a threshold of 3 standard deviations.<h3>

<h5> - 99.7% of data should fall within 3 standard deviations. By removing any outliers, a much more consistent EDA will be obtained.<h5>
<h5> - 'waterfront' and 'yr_renovated' are not included in outlier analysis/removal. As only 6 properties are waterfront properties and a small number of properties have been renovated, these properties would be removed if included and could provide valuable insights.<h5>

In [5]:
from scipy.stats import zscore
import pandas as pd

# Step 1: Remove non-numeric columns, including 'date' which is datetime64[ns]
non_numeric_cols = df[['id', 'date', 'yr_renovated', 'zipcode', 'lat', 'long', 'view', 'waterfront']]
df_numeric = df.drop(columns=['id', 'date', 'yr_renovated', 'zipcode', 'lat', 'long', 'view', 'waterfront'])

# Step 2: Perform outlier removal using z-scores on the numeric columns
threshold = 3
for _ in range(6):  # Repeat 6 times
    # Calculate z-scores for numerical columns
    df_zscores = df_numeric.apply(zscore)
    
    # Identify outliers based on the z-score threshold
    outliers = (df_zscores > threshold) | (df_zscores < -threshold)
    
    # Identify rows with any outliers
    outliers_any = outliers.any(axis=1)
    
    # Filter out the rows with outliers
    df_numeric = df_numeric[~outliers_any]

# Step 3: Rejoin the non-numeric columns with the cleaned numeric DataFrame
df_cleaned = pd.concat([df_numeric.reset_index(drop=True), non_numeric_cols.loc[df_numeric.index].reset_index(drop=True)], axis=1)

# Step 4: Remove any duplicate columns if they exist (shouldn't occur with this approach)
df_cleaned = df_cleaned.loc[:, ~df_cleaned.columns.duplicated()]

# Step 5: Display the cleaned DataFrame
print(df_cleaned)


          price  bedrooms  bathrooms  sqft_living  sqft_lot  floors  \
0      993000.0         4       2.00         2850     14810     2.0   
1      991700.0         4       3.00         2290      2350     2.0   
2      990400.0         3       2.50         2100      4097     2.0   
3      990000.0         3       2.75         2500      6350     2.0   
4      990000.0         4       3.75         3450      4940     2.0   
...         ...       ...        ...          ...       ...     ...   
16370   86500.0         3       1.00          840      9480     1.0   
16371   85000.0         2       1.00          830      9000     1.0   
16372   85000.0         2       1.00          910      9753     1.0   
16373   83000.0         2       1.00          900      8580     1.0   
16374   82000.0         3       1.00          860     10426     1.0   

       condition  grade  sqft_above  sqft_basement  ...  sqft_living15  \
0              5      8        2490            360  ...           2430   

In [6]:
df_cleaned['month'] = df_cleaned['date'].dt.month
df_cleaned['decade_built'] = (df_cleaned['yr_built']//10) * 10

In [9]:
df_cleaned = df_cleaned.drop(columns = ['id'])

In [10]:
df_cleaned.to_csv('/workspaces/Practise-Code/Kaggle Project/Data/Cleaned Data/df_cleaned.csv')

df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16375 entries, 0 to 16374
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   price          16375 non-null  float64       
 1   bedrooms       16375 non-null  int64         
 2   bathrooms      16375 non-null  float64       
 3   sqft_living    16375 non-null  int64         
 4   sqft_lot       16375 non-null  int64         
 5   floors         16375 non-null  float64       
 6   condition      16375 non-null  int64         
 7   grade          16375 non-null  int64         
 8   sqft_above     16375 non-null  int64         
 9   sqft_basement  16375 non-null  int64         
 10  yr_built       16375 non-null  int64         
 11  sqft_living15  16375 non-null  int64         
 12  sqft_lot15     16375 non-null  int64         
 13  date           16375 non-null  datetime64[ns]
 14  yr_renovated   16375 non-null  int64         
 15  zipcode        1637